# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# The path to our CSV file
file = ("./output_data/cities.csv")

# Read CSV weather data into pandas
weather_data = pd.read_csv(file)
clean_data = weather_data.drop(['Unnamed: 0'],axis=1)
clean_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,sola,-13.88,167.55,80.60,88,75,19.04,VU,1602518561
1,saldanha,-33.01,17.94,71.60,35,61,12.75,ZA,1602518374
2,inhambane,-23.86,35.38,69.80,68,75,2.24,MZ,1602518410
3,zemio,5.03,25.14,75.67,83,100,1.77,CF,1602518517
4,manado,1.49,124.85,77.00,94,20,3.67,ID,1602518561
...,...,...,...,...,...,...,...,...,...
570,wewak,-3.55,143.63,80.38,85,89,8.12,PG,1602518607
571,artyom,40.47,50.33,62.60,63,40,9.17,AZ,1602518607
572,saint-ambroise,48.55,-71.33,41.00,65,20,3.36,CA,1602518607
573,aksum,14.12,38.72,64.00,53,82,2.68,ET,1602518607


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key = g_key)



In [8]:
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

fig = gmaps.figure()
locations = clean_data[['Lat','Lng']].astype(float)
humidity = clean_data["Humidity"]

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100,point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#Narrow down the DataFrame to find your ideal weather condition
vacation_cities_df = clean_data.loc[(clean_data["Wind Speed"] <= 10) & (clean_data["Cloudiness"] <= 0) & (clean_data["Max Temp"] >= 80) & (clean_data["Max Temp"] <= 90)].dropna()
vacation_cities_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
253,aden,12.78,45.04,85.03,61,0,3.36,YE,1602518492
260,buin,-33.73,-70.75,80.01,27,0,4.70,CL,1602518582
293,kathor,21.30,72.93,82.40,69,0,5.82,IN,1602518585
309,guaratuba,-25.88,-48.57,80.01,60,0,5.01,BR,1602518321
354,irbil,36.19,44.01,82.40,11,0,4.70,IQ,1602518357
520,salalah,17.02,54.09,80.60,83,0,3.36,OM,1602518376
552,akyab,20.15,92.90,82.85,76,0,1.19,MM,1602518398
574,caborca,30.58,-111.10,83.39,18,0,4.94,MX,1602518607


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Store into variable named hotel_df
hotel_df = vacation_cities_df.loc[:,["City","Country", "Lat", "Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Lat,Lng,Hotel Name
253,aden,YE,12.78,45.04,
260,buin,CL,-33.73,-70.75,
293,kathor,IN,21.30,72.93,
309,guaratuba,BR,-25.88,-48.57,
354,irbil,IQ,36.19,44.01,
520,salalah,OM,17.02,54.09,
552,akyab,MM,20.15,92.90,
574,caborca,MX,30.58,-111.10,


In [11]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"radius" : 5000,
          "key" : g_key,
          "type" : "hotel",
          "keyword" : "hotel"}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    print(f"Retrieving results for hotels near vacation cities {index}: {city_name}.")
    response = requests.get(base_url, params = params).json()
    
    results = response['results']
    
    try:
        print(f"The closest hotel to {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available
    except (KeyError, IndexError):
        print("No hotel available near Vacation City... skipping.")
        

print("--Finished searching for hotels near Vacation Cities--")

Retrieving results for hotels near vacation cities 253: aden.
The closest hotel to aden is Coral Hotel Aden.
Retrieving results for hotels near vacation cities 260: buin.
The closest hotel to buin is Paihuen.
Retrieving results for hotels near vacation cities 293: kathor.
The closest hotel to kathor is Sapphero Harshraj Resort.
Retrieving results for hotels near vacation cities 309: guaratuba.
The closest hotel to guaratuba is Hotel VillaReal - Praia Central.
Retrieving results for hotels near vacation cities 354: irbil.
The closest hotel to irbil is Divan Erbil Hotel.
Retrieving results for hotels near vacation cities 520: salalah.
The closest hotel to salalah is Al Haffah House.
Retrieving results for hotels near vacation cities 552: akyab.
The closest hotel to akyab is Yuzana Aung Hotel.
Retrieving results for hotels near vacation cities 574: caborca.
The closest hotel to caborca is Hotel Elba.
--Finished searching for hotels near Vacation Cities--


In [12]:
#Store the hotel results into the DataFrame.
hotel_df = hotel_df.loc[hotel_df['Hotel Name']!='']
hotel_df.head()




,City,Country,Lat,Lng,Hotel Name
253,aden,YE,12.78,45.04,Coral Hotel Aden
260,buin,CL,-33.73,-70.75,Paihuen
293,kathor,IN,21.30,72.93,Sapphero Harshraj Resort
309,guaratuba,BR,-25.88,-48.57,Hotel VillaReal - Praia Central
354,irbil,IQ,36.19,44.01,Divan Erbil Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
#Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))